<a href="https://colab.research.google.com/github/hrnrhty/my-vae-nnabla/blob/main/step1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1 - VAE の学習 (MNIST)

最初のステップでは、[nnabla-examples](https://github.com/sony/nnabla-examples) に含まれる VAE (Variational Auto Encoder) をそのまま実行し、学習を行います。学習に用いるデータセットには、手書き数字のデータセットである [MNIST](http://yann.lecun.com/exdb/mnist/) が使用されています。

このノートブックでは、学習時間短縮のため GPU アクセラレーションを有効にしています。もし、以降のコードがうまく動かなかった場合は、タブメニューの「ランタイム」 > 「ランタイムのタイプを変更」をクリックし、「ハードウェア アクセラレータ」のドロップダウンメニューで「GPU」が選択されているか確認してください。「GPU」が選択されていなかった場合は、「GPU」を選択して「保存」ボタンをクリックしてください。

## Google Drive のマウント

学習結果の永続化のため Google Drive をマウントします。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## NNabla のインストール

Sony の [Neural Network Libraries](https://github.com/sony/nnabla) (NNabla) は Google Colab のランタイムにインストールされていないため、インストールします。ここでは、GPU アクセラレーションが使用できるよう、CUDA 版をインストールします。

NNabla の依存パッケージの中には既に Google Colab にインストールされているものがあり、NNabla が要求するバージョンと一致していない場合インストール時にエラーが発生することがあります。エラーを無視して先に進んでも問題なく動作することが多いですが、ここではエラー発生を回避します。具体的には、仮想環境を作成し、その中に NNabla をインストールしていきます。

通常、仮想環境の作成には Python 3.3 以降標準パッケージに含まれるようになった `venv` が使用されますが、Google Colab ではうまく動作しません。そこで、代わりに [virtualenv](https://virtualenv.pypa.io/en/latest/) をインストールします。`virtualenv` は `venv` の元になったパッケージです。

まず、`virtualenv` をインストールし、続いて `venv` という名称の仮想環境を作成します。

In [ ]:
!pip install virtualenv
!virtualenv venv

仮想環境をアクティベートし、仮想環境内に NNabla (CUDA版) をインストールします。2022 年 2 月 2 日時点、最新版である v1.25.0 をインストールします。

ここで、`!` によるシェルコマンドはサブプロセスとして実行されるため、コマンド実行終了後は状態が維持されないことに注意が必要です。`&&` で連結し1つのコマンドとしてシェルに渡しましょう。2 行に分けてしまうと、2 行目は仮想環境がアクティベートされていない別のサブプロセスになるため、NNabla が仮想環境外にインストールされてしまいます。

In [ ]:
!source venv/bin/activate && pip install nnabla-ext-cuda100==1.25.0

## nnabla-examples のクローン

[nnabla-examples](https://github.com/sony/nnabla-examples) は活発にメンテナンスされており、時々ディレクトリ構成が変更されるため、ここでは Tag v1.25.0 を指定してクローンします。

In [ ]:
!if [ ! -d nnabla-examples ]; then git clone https://github.com/sony/nnabla-examples.git -b v1.25.0 --depth 1; fi

## 学習の実行

GPU アクセラレーションを有効にするため、`-c cudnn` オプションつきで `vae.py` を実行します。GPU を利用することで、学習にかかる時間が飛躍的に短縮されます。試しに CPU のみで実行したときの学習速度を体感してみたい場合は、`-c cudnn` を削除して実行してください。

また、ここでも、仮想環境のアクティベートを忘れないようにしましょう。

In [ ]:
!source venv/bin/activate && cd nnabla-examples/image-classification/mnist-collection && python vae.py -c cudnn

## 学習結果を Google Drive に保存

In [ ]:
!cd drive/MyDrive && if [ ! -d my-vae-nnabla ]; then mkdir my-vae-nnabla; fi
!cd drive/MyDrive/my-vae-nnabla && if [ -d step1 ]; then rm -rf step1; fi
!cp -r nnabla-examples/image-classification/mnist-collection/tmp.monitor.vae drive/MyDrive/my-vae-nnabla/step1

Google Drive に保存した学習結果は、Step 2 で使用します。